In [4]:
import numpy as np
from math import pi
from scipy.fft import fft, ifft

In [5]:
x = [1, -2, 2, 4]

In [30]:
def conv_circular(x, h, verbose=False):
    N = len(x)
    M = len(h)
    w = np.zeros(N)
    for k in range(N):
        if verbose: print(f'w[{k}] = ', end=' ')
        for m in range(M):
            if verbose: print(f'h[{m}] * x[{k-m}] ({h[m]}*{x[k-m]}={h[m] * x[k-m]})', end= '')
            if verbose and m+1 < M: print(' + ', end='')
            w[k] += h[m] * x[k-m]
        if verbose: print(f' = {w[k]}')
    return w

def test_conv_cicrular(N=100):
    x = np.random.rand(N)
    h = np.random.rand(5)
    
    golden = (ifft(fft(x,N) * fft(h,N)))
    dut = conv_circular(x,h)
    
    assert np.allclose(golden, dut), 'Teste falhou'
    print('Teste passou')
    
test_conv_cicrular()

Teste passou


In [9]:
def haar_one_stage_analysis(x): 
    N = len(x)
    l_analysis = [0.5, 0.5]
    h_analysis = [0.5, -0.5]
    
    Xl = conv_circular(x,l_analysis)[::2]
    Xh = conv_circular(x,h_analysis)[::2]
    
    X = np.concatenate([Xl, Xh])
    return X, Xl, Xh

In [10]:
X, Xl, Xh = haar_one_stage_analysis(x)
X

array([ 2.5,  0. , -1.5,  2. ])

In [12]:
def upsample(x): return np.concatenate([np.insert(x, slice(1, None), 0), np.zeros(1)])

def haar_one_stage_synthesis(X):
    N = len(X)
    # transformado em filtro causal 
    l_synthesis = [1, 1]
    h_synthesis = [-1, 1]
    
    Xl_condensed = X[:N//2]
    Xh_condensed = X[N//2:]
    Xl = upsample(Xl_condensed)
    Xh = upsample(Xh_condensed)
    
    vl = conv_circular(Xl, l_synthesis)
    vh = conv_circular(Xh, h_synthesis)
    
    x_delayed = vl + vh
    x = np.roll(x_delayed, -1)
    
    return x


In [13]:
haar_one_stage_synthesis(X)

array([ 1., -2.,  2.,  4.])

### Ok , tenho o resultado para comparar as contas na mão

fazendo agora passo a passo

In [14]:
S1_ls = [1, 1]
U_Xl = upsample(Xl)
U_Xl

array([2.5, 0. , 0. , 0. ])

In [31]:
vl = conv_circular(U_Xl, S1_ls, verbose=True)
vl

w[0] =  h[0] * x[0] (1*2.5=2.5) + h[1] * x[-1] (1*0.0=0.0) = 2.5
w[1] =  h[0] * x[1] (1*0.0=0.0) + h[1] * x[0] (1*2.5=2.5) = 2.5
w[2] =  h[0] * x[2] (1*0.0=0.0) + h[1] * x[1] (1*0.0=0.0) = 0.0
w[3] =  h[0] * x[3] (1*0.0=0.0) + h[1] * x[2] (1*0.0=0.0) = 0.0


array([2.5, 2.5, 0. , 0. ])

In [18]:
S1_hs = [-1, 1]
U_Xh = upsample(Xh)
U_Xh

array([-1.5,  0. ,  2. ,  0. ])

In [33]:
vh = conv_circular(U_Xh, S1_hs, verbose=True)
vh

w[0] =  h[0] * x[0] (-1*-1.5=1.5) + h[1] * x[-1] (1*0.0=0.0) = 1.5
w[1] =  h[0] * x[1] (-1*0.0=-0.0) + h[1] * x[0] (1*-1.5=-1.5) = -1.5
w[2] =  h[0] * x[2] (-1*2.0=-2.0) + h[1] * x[1] (1*0.0=0.0) = -2.0
w[3] =  h[0] * x[3] (-1*0.0=-0.0) + h[1] * x[2] (1*2.0=2.0) = 2.0


array([ 1.5, -1.5, -2. ,  2. ])

In [21]:
S1_x = vl + vh
S1_x

array([ 4.,  1., -2.,  2.])

In [22]:
x = np.roll(S1_x, -1)
x

array([ 1., -2.,  2.,  4.])